Carga de datos en distintos formatos
===

* *30 min* | Última modificación: Junio 22, 2019

Hive permite la lectura de archivos en distintos formatos. Al finalizar este tutorial, el lector estará en capacidad de leer archivos en formatos de texto, CSV y JSON.

**Preparación**

In [1]:
##
## Se carga la librería para interactuar con Hive desde Jupyter.
## 
%load_ext bigdata
%timeout 300

In [2]:
##
## Crea la carpeta drivers en el HDFS
##
!hdfs dfs -rm -r -f /tmp/drivers
!hdfs dfs -mkdir    /tmp/drivers

Deleted /tmp/drivers


## Lectura de formato JSON desde un archivo cargado como texto

En esta estrategía de carga de datos, el archivo original con formato JSON es cargado como texto, donde cada registro de la tabla corresponde a una línea del archivo original.

In [3]:
##
## Copia el archivo al HDFS para su importación posterior a Hive
##
!hdfs dfs -copyFromLocal drivers/drivers.json  /tmp/drivers/drivers.json

En la siguiente celda, se crea la tabla `drivers_raw_json`, la cual tiene una única columna llamada `textcol`. Luego, el archivo `drivers.json` es cargado en dicha tabla. Finalmente, se visualizan los primeros cinco registros para verificar que la lectura fue correcta.

In [4]:
%%hive
DROP TABLE IF EXISTS drivers_raw_json;

CREATE TABLE drivers_raw_json (
    textcol STRING
) 
STORED AS TEXTFILE;

LOAD DATA INPATH 
    '/tmp/drivers/drivers.json' 
OVERWRITE INTO TABLE drivers_raw_json;

SELECT * FROM drivers_raw_json LIMIT 5;

DROP TABLE IF EXISTS drivers_raw_json;
OK
Time taken: 8.193 seconds
CREATE TABLE drivers_raw_json (
    textcol STRING
) 
STORED AS TEXTFILE;
OK
Time taken: 0.557 seconds
LOAD DATA INPATH 
    '/tmp/drivers/drivers.json' 
OVERWRITE INTO TABLE drivers_raw_json;
Loading data to table default.drivers_raw_json
OK
Time taken: 0.889 seconds
SELECT * FROM drivers_raw_json LIMIT 5;
OK
{"driverId":10,"name":"George Vetticaden","ssn":621011971,"location":"244-4532 Nulla Rd.","certified":"N","wage-plan":"miles"}
{"driverId":11,"name":"Jamie Engesser","ssn":262112338,"location":"366-4125 Ac Street","certified":"N","wage-plan":"miles"}
{"driverId":12,"name":"Paul Coddin","ssn":198041975,"location":"Ap #622-957 Risus. Street","certified":"Y","wage-plan":"hours"}
{"driverId":13,"name":"Joe Niemiec","ssn":139907145,"location":"2071 Hendrerit. Ave","certified":"Y","wage-plan":"hours"}
{"driverId":14,"name":"Adis Cesir","ssn":820812209,"location":"Ap #810-1228 In St.","certified":"Y","wage-plan":"hours"

Note que en la salida anterior, cada fila corresponde a un registro.

### Lectura usando get_json_object

Los valores de los campos pueden ser extraídos usando la función `get_json_object`, cuyos parámetros son el nombre del campo en la tabla y el nombre del campo en la estructura JSON. En el siguiente fragmento de código, se utiliza una consulta para extraer los campos `driverId`, `name`  y `ssn`.

In [5]:
%%hive
SELECT
    GET_JSON_OBJECT(textcol,'$.driverId'),
    GET_JSON_OBJECT(textcol,'$.name'),
    GET_JSON_OBJECT(textcol,'$.ssn')
FROM 
    drivers_raw_json 
LIMIT 
    10;

SELECT
    GET_JSON_OBJECT(textcol,'$.driverId'),
    GET_JSON_OBJECT(textcol,'$.name'),
    GET_JSON_OBJECT(textcol,'$.ssn')
FROM 
    drivers_raw_json 
LIMIT 
    10;
OK
10	George Vetticaden	621011971
11	Jamie Engesser	262112338
12	Paul Coddin	198041975
13	Joe Niemiec	139907145
14	Adis Cesir	820812209
15	Rohit Bakshi	239005227
16	Tom McCuch	363303105
17	Eric Mizell	123808238
18	Grant Liu	171010151
19	Ajay Singh	160005158
Time taken: 0.361 seconds, Fetched: 10 row(s)


### Uso de json_tuple

Esta función cumple el mismo objetivo de la anterior, pero es mucho más eficiente ya que el registro es procesado únicamente una vez para realizar la extracción de la información requerida. Ya que `json_tuple` es una UDF, debe usarse `LATERAL VIEW` para realizar la consulta, tal como se ejemplifica a continuación.

In [6]:
%%hive
SELECT
    t1.driverId,
    t1.name,
    t1.ssn
FROM
    drivers_raw_json  t0
LATERAL VIEW
    JSON_TUPLE(t0.textcol, 'driverId', 'name', 'ssn') t1
    AS driverId, name, ssn
LIMIT 5;

SELECT
    t1.driverId,
    t1.name,
    t1.ssn
FROM
    drivers_raw_json  t0
LATERAL VIEW
    JSON_TUPLE(t0.textcol, 'driverId', 'name', 'ssn') t1
    AS driverId, name, ssn
LIMIT 5;
OK
10	George Vetticaden	621011971
11	Jamie Engesser	262112338
12	Paul Coddin	198041975
13	Joe Niemiec	139907145
14	Adis Cesir	820812209
Time taken: 0.079 seconds, Fetched: 5 row(s)


## Carga de archivos en formato JSON

Hive también permite la importación directa de archivos en formato JSON usando el serde `JsonSerDe`.

In [7]:
##
## Se copia el archivo al sistema HDFS
##
!hdfs dfs -copyFromLocal drivers/drivers.json  /tmp/drivers/drivers.json

En la siguiente celda, se crea la tabla `drivers_json` donde el formato de cada registro es especificado como JSON

In [8]:
%%hive
DROP TABLE IF EXISTS drivers_json;

CREATE TABLE drivers_json (
    driverId  INT, 
    name      STRING, 
    ssn       BIGINT,
    location  STRING, 
    certified STRING, 
    wageplan  STRING)
ROW FORMAT SERDE 'org.apache.hive.hcatalog.data.JsonSerDe'
STORED AS TEXTFILE
LOCATION '/tmp/drivers-json';

LOAD DATA INPATH '/tmp/drivers/drivers.json' OVERWRITE INTO TABLE drivers_json;

SELECT * FROM drivers_json LIMIT 5;

DROP TABLE IF EXISTS drivers_json;
OK
Time taken: 0.083 seconds
CREATE TABLE drivers_json (
    driverId  INT, 
    name      STRING, 
    ssn       BIGINT,
    location  STRING, 
    certified STRING, 
    wageplan  STRING)
ROW FORMAT SERDE 'org.apache.hive.hcatalog.data.JsonSerDe'
STORED AS TEXTFILE
LOCATION '/tmp/drivers-json';
OK
Time taken: 0.074 seconds
json;
Loading data to table default.drivers_json
OK
Time taken: 0.559 seconds
SELECT * FROM drivers_json LIMIT 5;
OK
10	George Vetticaden	621011971	244-4532 Nulla Rd.	N	NULL
11	Jamie Engesser	262112338	366-4125 Ac Street	N	NULL
12	Paul Coddin	198041975	Ap #622-957 Risus. Street	Y	NULL
13	Joe Niemiec	139907145	2071 Hendrerit. Ave	Y	NULL
14	Adis Cesir	820812209	Ap #810-1228 In St.	Y	NULL
Time taken: 0.116 seconds, Fetched: 5 row(s)


## Carga de archivos en formato CSV

En este ejemplo se usa el serde `OpenCSVSerde` para leer archivo en formato CSV. Note que se usa la cláusula `with serdeproperties` para indicar las características del formato CSV utilizado.

In [9]:
##
## Se copia el archivo al sistema HDFS
##
!hdfs dfs -copyFromLocal drivers/drivers.csv  /tmp/drivers/drivers.csv

In [10]:
%%hive
DROP TABLE IF EXISTS drivers_csv;

CREATE TABLE drivers_csv (driverId  INT, 
                         name      STRING, 
                         ssn       BIGINT,
                         location  STRING, 
                         certified STRING, 
                         wageplan  STRING)
ROW FORMAT SERDE 
    'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
   'separatorChar' = ",",
   'quoteChar'     = '\'',
   'escapeChar'    = "\\");

LOAD DATA INPATH '/tmp/drivers/drivers.csv' OVERWRITE INTO TABLE drivers_csv;

SELECT * FROM drivers_csv LIMIT 5;

DROP TABLE IF EXISTS drivers_csv;
OK
Time taken: 0.065 seconds
CREATE TABLE drivers_csv (driverId  INT, 
                         name      STRING, 
                         ssn       BIGINT,
                         location  STRING, 
                         certified STRING, 
                         wageplan  STRING)
ROW FORMAT SERDE 
    'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
   'separatorChar' = ",",
   'quoteChar'     = '\'',
   'escapeChar'    = "\\");
OK
Time taken: 0.046 seconds
sv;
Loading data to table default.drivers_csv
OK
Time taken: 0.483 seconds
SELECT * FROM drivers_csv LIMIT 5;
OK
driverId	name	ssn	location	certified	wage-plan
10	George Vetticaden	621011971	244-4532 Nulla Rd.	N	miles
11	Jamie Engesser	262112338	366-4125 Ac Street	N	miles
12	Paul Coddin	198041975	Ap #622-957 Risus. Street	Y	hours
13	Joe Niemiec	139907145	2071 Hendrerit. Ave	Y	hours
Time taken: 0.107 seconds, Fetched: 5 row(s)


## Carga usando expresiones regulaes y RegexSerDe

En este caso, se realiza la carga de datos especificando los campos mediante el uso de expresiones regulares.

In [11]:
##
## Copia los archivos al HDFS
##
!hdfs dfs -copyFromLocal drivers/drivers.csv  /tmp/drivers/drivers.csv

In [12]:
%%hive
DROP TABLE IF EXISTS drivers_regex;

CREATE TABLE drivers_regex(
    driverId  INT, 
    name      STRING, 
    ssn       BIGINT,
    location  STRING, 
    certified STRING, 
    wageplan  STRING)
ROW FORMAT SERDE 
    'org.apache.hadoop.hive.serde2.RegexSerDe'
WITH SERDEPROPERTIES (
   'input.regex' = '(\\d+),([^,]*),(\\d+),([^,]*),([^,]*),([^,]*)')
TBLPROPERTIES ("skip.header.line.count"="1");

LOAD DATA INPATH '/tmp/drivers/drivers.csv' OVERWRITE INTO TABLE drivers_regex;

SELECT * FROM drivers_regex LIMIT 5;

DROP TABLE IF EXISTS drivers_regex;
OK
Time taken: 0.052 seconds
CREATE TABLE drivers_regex(
    driverId  INT, 
    name      STRING, 
    ssn       BIGINT,
    location  STRING, 
    certified STRING, 
    wageplan  STRING)
ROW FORMAT SERDE 
    'org.apache.hadoop.hive.serde2.RegexSerDe'
WITH SERDEPROPERTIES (
   'input.regex' = '(\\d+),([^,]*),(\\d+),([^,]*),([^,]*),([^,]*)')
TBLPROPERTIES ("skip.header.line.count"="1");
OK
Time taken: 0.049 seconds
egex;
Loading data to table default.drivers_regex
OK
Time taken: 0.438 seconds
SELECT * FROM drivers_regex LIMIT 5;
OK
10	George Vetticaden	621011971	244-4532 Nulla Rd.	N	miles
11	Jamie Engesser	262112338	366-4125 Ac Street	N	miles
12	Paul Coddin	198041975	Ap #622-957 Risus. Street	Y	hours
13	Joe Niemiec	139907145	2071 Hendrerit. Ave	Y	hours
14	Adis Cesir	820812209	Ap #810-1228 In St.	Y	hours
Time taken: 0.102 seconds, Fetched: 5 row(s)


La expresión regular usada es la siguiente: 

    (\\d+),([^,]*),(\\d+),([^,]*),([^,]*),([^,]*)`

donde:

* Los paréntesis indica los campos, esto es, `(\\d+)` es el primer campo, `([^,]*)` es el segundo y así sucesivamente.

* Se indica que la coma es el separador entre campos.

* `(\\d+)` representa una cadena de uno o más dígitos.

* `[...]` representan uno o más posibles caracteres, de tal forma que `[^,]` indica cualquier caracter excepto una coma. Finalmente, el * indica cero, una o más ocurrencias. Es así como `[^,]*` representa cualquier cadena de caracteres que no contenga una coma.

* La expresión regular usada indica que los campos 1 y 3 son numéricos, y los restantes son texto.